In [ ]:
# 1.load Dataset
# 2 Enbedding
#   Vocabulary
#   make vectors
# 3.Build RNN
# 4.prediction

In [ ]:
import pandas as pd
df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100
...,...,...
85,Who directed the movie 'Titanic'?,JamesCameron
86,Which superhero is also known as the Dark Knight?,Batman
87,What is the capital of Brazil?,Brasilia
88,Which fruit is known as the king of fruits?,Mango


In [ ]:
# Tokenize
def Tokenize(text):
  text = text.lower()
  text = text.replace('?',"")
  text = text.replace("'","")
  return text.split()

print(Tokenize('What is the capital of France?'))

['what', 'is', 'the', 'capital', 'of', 'france']


In [ ]:
# Vocab.
vocab = {'<UNK>': 0}

In [ ]:
def build_vocab(row):
  print(row['question'],row['answer'])
  Tokenized_q = Tokenize(row['question'])
  Tokenized_A = Tokenize(row['answer'])

  merged_Token = Tokenized_q + Tokenized_A
  for token in merged_Token:
    if token not in vocab:
      vocab[token] = len(vocab)

In [ ]:
df.apply(build_vocab,axis = 1)

What is the capital of France? Paris
What is the capital of Germany? Berlin
Who wrote 'To Kill a Mockingbird'? Harper-Lee
What is the largest planet in our solar system? Jupiter
What is the boiling point of water in Celsius? 100
Who painted the Mona Lisa? Leonardo-da-Vinci
What is the square root of 64? 8
What is the chemical symbol for gold? Au
Which year did World War II end? 1945
What is the longest river in the world? Nile
What is the capital of Japan? Tokyo
Who developed the theory of relativity? Albert-Einstein
What is the freezing point of water in Fahrenheit? 32
Which planet is known as the Red Planet? Mars
Who is the author of '1984'? George-Orwell
What is the currency of the United Kingdom? Pound
What is the capital of India? Delhi
Who discovered gravity? Newton
How many continents are there on Earth? 7
Which gas do plants use for photosynthesis? CO2
What is the smallest prime number? 2
Who invented the telephone? Alexander-Graham-Bell
What is the capital of Australia? Canber

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [ ]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [ ]:
len(vocab)

324

In [ ]:
# Convert to Numerical Indices
def text_to_indices(text,vocab):
  indexed_text = []
  for token in Tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])
  return indexed_text

In [ ]:
text_to_indices('What is the capital of France?',vocab)

[1, 2, 3, 4, 5, 6]

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class QADataset(Dataset):
  def __init__(self,df,vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self,idx):
    numerical_q = text_to_indices(self.df.iloc[idx]['question'],self.vocab)
    numerical_ans = text_to_indices(self.df.iloc[idx]['answer'],self.vocab)
    return torch.tensor(numerical_q),torch.tensor(numerical_ans)

In [ ]:
dataset = QADataset(df,vocab)

In [ ]:
dataset[10]

(tensor([ 1,  2,  3,  4,  5, 53]), tensor([54]))

In [ ]:
dataloader = DataLoader(dataset,batch_size=1,shuffle=True)

In [ ]:
for ques, ans in dataloader:
  print(ques,ans)

tensor([[  1,   2,   3, 103,   5, 104,  19, 105]]) tensor([[106]])
tensor([[  1,   2,   3,   4,   5, 279]]) tensor([[280]])
tensor([[10, 55,  3, 56,  5, 57]]) tensor([[58]])
tensor([[ 1,  2,  3, 69,  5, 53]]) tensor([[260]])
tensor([[ 10,  11, 157, 158, 159]]) tensor([[160]])
tensor([[  1,   2,   3,  69,   5, 155]]) tensor([[156]])
tensor([[ 1,  2,  3, 92, 93, 94]]) tensor([[95]])
tensor([[ 10,  75, 208]]) tensor([[209]])
tensor([[  1,   2,   3, 122, 123,  19,   3,  45]]) tensor([[124]])
tensor([[ 78,  79, 288,  81,  19,  14, 289]]) tensor([[85]])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([[114]])
tensor([[ 42,  18, 118,   3, 186, 187]]) tensor([[188]])
tensor([[ 42, 101,   2,   3,  17]]) tensor([[102]])
tensor([[ 10, 140,   3, 141, 270,  93, 271,   5,   3, 272]]) tensor([[273]])
tensor([[ 42,  18,   2,   3, 281,  12,   3, 282]]) tensor([[205]])
tensor([[ 1,  2,  3, 59, 25,  5, 26, 19, 60]]) tensor([[61]])
tensor([[  1,   2,   3,   4,   5, 286]]) tensor([[287]])
tensor([[  1,   2

In [ ]:
# RNN Architecture
# input_neuron = 50(Enbeddings), hidden = 64, output_neuron = 324

In [ ]:
import torch.nn as nn

In [ ]:
class SimpleRNN(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,embedding_dim = 50)  # it is converting every value of tensor into 50 dimentional tensor.
    self.rnn = nn.RNN(50,64,batch_first = True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, t):
    t = self.embedding(t)
    hidden,final = self.rnn(t)
    output = self.fc(final.squeeze(0))
    return output

ROugh WOrk

In [ ]:
dataset[3][0]

tensor([ 1,  2,  3, 17, 18, 19, 20, 21, 22])

In [ ]:
x = nn.Embedding(324,embedding_dim = 50)

In [ ]:
a = x(dataset[3][0])

In [ ]:
a.shape

torch.Size([9, 50])

In [ ]:
b = dataset[3][1]

In [ ]:
b

tensor([23])

In [ ]:
y = nn.RNN(50,64)

In [ ]:
p = y(a)[0] # combination of all outputs including hidden layers
p.shape

torch.Size([9, 64])

In [ ]:
 q = y(a)[1] # Only last layers
 q.shape

torch.Size([1, 64])

In [ ]:
z = nn.Linear(64,324)

In [ ]:
z(q)

tensor([[ 0.1304, -0.1933, -0.0630,  0.1019,  0.2892,  0.4015,  0.0613,  0.3899,
         -0.3861,  0.0856,  0.0309, -0.3484, -0.3111, -0.4029,  0.1571,  0.1319,
         -0.1672, -0.3076,  0.1086, -0.4885,  0.0289, -0.1691,  0.1352,  0.0529,
          0.1213, -0.7542, -0.2947,  0.3545, -0.4653, -0.3311,  0.4256, -0.2277,
          0.0188, -0.0746, -0.2069, -0.0234, -0.0108,  0.0143, -0.2180, -0.2659,
          0.1264,  0.4996, -0.4792, -0.3897,  0.0455, -0.0634,  0.1971, -0.2866,
          0.0010, -0.2201, -0.2846, -0.3124,  0.2656,  0.2351,  0.3337,  0.2331,
         -0.3388,  0.0086,  0.0025,  0.0362,  0.2975,  0.1832, -0.0988, -0.1083,
          0.2974, -0.4932,  0.1953, -0.0119, -0.3621, -0.3112,  0.0624,  0.4793,
         -0.0450,  0.0424,  0.2732, -0.2160, -0.3760, -0.2115,  0.4238, -0.7628,
          0.2488,  0.0564, -0.0865, -0.0968, -0.0455,  0.3229,  0.3064, -0.4472,
          0.1228, -0.1218, -0.2283,  0.3132,  0.0564,  0.4213, -0.2977, -0.0577,
          0.0237,  0.3932, -

In [ ]:
z(q).shape

torch.Size([1, 324])

In [ ]:
learning_rate = 0.001
epochs =  20

In [ ]:
model = SimpleRNN(len(vocab))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [ ]:
#training Loop
for epoch in range(epochs):
  total_loss = 0
  for question, answer in dataloader:
    optimizer.zero_grad()
    output = model(question)
    loss = criterion(output, answer.squeeze(0))
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  print(f"Epoch {epoch+1} Loss: {total_loss:4f}")

Epoch 1 Loss: 521.958956
Epoch 2 Loss: 449.335357
Epoch 3 Loss: 371.007357
Epoch 4 Loss: 313.370017
Epoch 5 Loss: 263.104739
Epoch 6 Loss: 215.431132
Epoch 7 Loss: 172.597504
Epoch 8 Loss: 133.848169
Epoch 9 Loss: 102.875833
Epoch 10 Loss: 78.552051
Epoch 11 Loss: 60.922851
Epoch 12 Loss: 47.736853
Epoch 13 Loss: 38.385776
Epoch 14 Loss: 31.191246
Epoch 15 Loss: 25.604124
Epoch 16 Loss: 21.374831
Epoch 17 Loss: 18.091788
Epoch 18 Loss: 15.334357
Epoch 19 Loss: 13.250129
Epoch 20 Loss: 11.425083


In [ ]:
def predict(model, question, threshold = 0.5):
  numerical_question = text_to_indices(question, vocab)
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)
  output = model(question_tensor)
  probabilities = torch.softmax(output, dim = 1)
  value, index = torch.max(probabilities,dim = 1)
  if(value < threshold):
    return "Don't Know"
  else:
    return list(vocab.keys())[index]

In [ ]:
predict(model,"What do bees collect from flowers?")

'nectar'